In [1]:
#load libraries
library(tidyverse)
library(dplyr)
library(lubridate)
library(ggplot2)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
#read in data
players <- read_csv("project_data/players.csv")
sessions <- read_csv("project_data/sessions.csv")

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [4]:
# Create a session_length column in minutes
sessions <- sessions |>
  mutate(
    start_time = ymd_hms(start_time),
    end_time = ymd_hms(end_time),
    session_length = as.numeric(difftime(end_time, start_time, units = "mins"))
  )
# Summarize for each player
player_activity_summary <- sessions |>
  group_by(hashedEmail) |>
  summarise(
    total_sessions = n(),
    total_playtime = sum(session_length, na.rm = TRUE),
    avg_session_length = mean(session_length, na.rm = TRUE)
  )

# Preview
player_activity_summary |> slice_head(n = 5)

hashedEmail,total_sessions,total_playtime,avg_session_length
<chr>,<int>,<dbl>,<dbl>
0088b5e134c3f0498a18c7ea6b8d77b4b0ff1636fc93355ccc95b45423367832,2,5.255778e+05,2.627889e+05
060aca80f8cfbf1c91553a72f4d5ec8034764b05ab59fe8e1cf0eee9a7b67967,1,5.000000e-01,5.000000e-01
0ce7bfa910d47fc91f21a7b3acd8f33bde6db57912ce0290fa0437ce0b97f387,1,1.833333e-01,1.833333e-01
0d4d71be33e2bc7266ee4983002bd930f69d304288a8663529c875f40f1750f3,13,6.966667e+00,5.358974e-01
0d70dd9cac34d646c810b1846fe6a85b9e288a76f5dcab9c1ff1a0e7ca200b3a,2,1.166667e+00,5.833333e-01


In [5]:
player_profiles <- players |>
  left_join(player_activity_summary, by = "hashedEmail")

# Preview
player_profiles |> 
    slice_head(n = 5)

experience,subscribe,hashedEmail,played_hours,name,gender,Age,total_sessions,total_playtime,avg_session_length
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,27,-1.310697e+07,-4.854432e+05
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17,3,4.250000e+00,1.416667e+00
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17,1,8.333333e-02,8.333333e-02
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21,1,8.333333e-01,8.333333e-01
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21,1,1.500000e-01,1.500000e-01


In [32]:
players_with_activity <- players |>
  left_join(player_summary, by = "hashedEmail")

# Display the first 5 rows to check the joined data
players_with_activity |>
  slice_head(n = 5)

experience,subscribe,hashedEmail,played_hours,name,gender,Age,total_sessions,total_playtime,avg_session_length
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,27,-1.310697e+07,-4.854432e+05
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17,3,4.250000e+00,1.416667e+00
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17,1,8.333333e-02,8.333333e-02
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21,1,8.333333e-01,8.333333e-01
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21,1,1.500000e-01,1.500000e-01


In [6]:
activity_features <- player_profiles |>
  select(total_sessions, total_playtime, avg_session_length) |>
  drop_na()

activity_features_scaled <- as_tibble(scale(activity_features))

activity_features_scaled |> 
    slice_head(n = 5)

total_sessions,total_playtime,avg_session_length
<dbl>,<dbl>,<dbl>
0.3561844,-7.220635764,-6.59581791
-0.2245511,-0.002835715,-0.09525277
-0.2729457,-0.002838010,-0.09527062
-0.2729457,-0.002837597,-0.09526058
-0.2729457,-0.002837973,-0.09526973


In [9]:
# Set a seed for reproducibility so clustering results are consistent each time the code runs
set.seed(123)
# Run k-means clustering with 3 clusters (as an example)
# Only cluster on the numeric, scaled features
kmeans_model <- kmeans(activity_features_scaled, centers = 3)

# Add the cluster assignments as a new column in the scaled feature table
activity_features_scaled <- activity_features_scaled |>
  mutate(cluster = as.factor(kmeans_model$cluster))

# Show the first 5 rows to preview cluster assignments
activity_features_scaled |> slice_head(n = 5)

total_sessions,total_playtime,avg_session_length,cluster
<dbl>,<dbl>,<dbl>,<fct>
0.3561844,-7.220635764,-6.59581791,2
-0.2245511,-0.002835715,-0.09525277,1
-0.2729457,-0.002838010,-0.09527062,1
-0.2729457,-0.002837597,-0.09526058,1
-0.2729457,-0.002837973,-0.09526973,1
